In [1]:
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
from scipy.stats import poisson

league = "usa"

leagueTable_url = f"https://www.soccerstats.com/homeaway.asp?league={league}"
headers = {'User-Agent': 'Mozilla/5.0'}  #helps prevent blocking by some servers
response = requests.get(leagueTable_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
if (response.status_code != 200):
    print(response.status_code) #error in scraping of data if code is not 200

hometableNum = 9
hometable = soup.find_all('table')[hometableNum] # find the home table on the webpage
hdf = pd.read_html(StringIO(str(hometable)))[0] # convert the html table into a DataFrame
awaytable = soup.find_all('table')[hometableNum+1] # find the home table on the webpage
adf = pd.read_html(StringIO(str(awaytable)))[0] # convert the html table into a DataFrame
if (len(adf.columns) == 8):
    hometableNum = 8 #table index of home table
elif (len(adf.columns) == 10):
    hometableNum = 9 #table index of home table
else:
    print("Problem finding the home/away tables")

hometable = soup.find_all('table')[hometableNum] # find the home table on the webpage
hdf = pd.read_html(StringIO(str(hometable)))[0] # convert the html table into a DataFrame
hdf = hdf.rename(columns={1: "Team", 2: "HMP", 6: "HGF", 7: "HGC"}) #rename columns
hdf = hdf.sort_values('Team') #sort A-Z
hdf = hdf.reset_index() #reset the index
hdf = hdf.drop(hdf.index[len(hdf)-1]) #drop bottom row
hdf = hdf.drop([0, 3, 4, 5, 8, 9, 10, 'index'], axis=1) #drop columns for league position, W, D, L, GD, PTS, and the index

awaytable = soup.find_all('table')[hometableNum+1] # find the home table on the webpage
adf = pd.read_html(StringIO(str(awaytable)))[0] # convert the html table into a DataFrame
adf = adf.rename(columns={1: "Team", 2: "AMP", 6: "AGF", 7: "AGC"}) #rename columns
adf = adf.sort_values('Team') #sort A-Z
adf = adf.reset_index() #reset the index
adf = adf.drop(adf.index[len(adf)-1]) #drop bottom row
adf = adf.drop([0, 3, 4, 5, 8, 9, 10, 'index'], axis=1) #drop columns for league position, W, D, L, GD, PTS, and the index

df = hdf.merge(adf) #merge into one dataframe
df = df[['Team', 'HMP', 'HGF', 'HGC', 'AMP', 'AGF', 'AGC']] #reorder columns
df[['HMP', 'HGF', 'HGC', 'AMP', 'AGF', 'AGC']] = df[['HMP', 'HGF', 'HGC', 'AMP', 'AGF', 'AGC']].apply(pd.to_numeric, errors='coerce') #convert columns to numerical
print(df)

               Team  HMP  HGF  HGC  AMP  AGF  AGC
0       Atlanta Utd   10   17   15    8    4   19
1            Austin   10    8    8    9    7   13
2       CF Montreal    7    5   15   11   10   18
3         Charlotte    8   17    9   10   12   20
4      Chicago Fire    7    7    9   10   28   22
5        Cincinnati    8   14    9   10   12   15
6   Colorado Rapids   10   11   14    8    7   11
7     Columbus Crew   10   17   11    8   12   14
8         DC United   10    9   17    9    8   21
9            Dallas    7    4    9   10   18   19
10   Houston Dynamo   10   13   19    8   10    9
11      Inter Miami    9   22   15    7   14   12
12        LA Galaxy    8    8   15   10   10   24
13   Los Angeles FC    8   15    5    8   15   18
14    Minnesota Utd    9   16   11    9   12    9
15     Nashville SC   10   21    9    8   11   12
16      New England    6    4    5   10   15   10
17    New York City   10   16    9    8    8   11
18      New York RB    9   23    8    9    6   13


In [2]:
df['HS'] = (df['HGF'] / df['HGF'].mean()).round(3) #home strength
df['HW'] = (df['HGC'] / df['HGC'].mean()).round(3) #home weakness
df['AS'] = (df['AGF'] / df['AGF'].mean()).round(3) #away strength
df['AW'] = (df['AGC'] / df['AGC'].mean()).round(3) #away weakness
print(df)

               Team  HMP  HGF  HGC  AMP  AGF  AGC     HS     HW     AS     AW
0       Atlanta Utd   10   17   15    8    4   19  1.200  1.324  0.353  1.341
1            Austin   10    8    8    9    7   13  0.565  0.706  0.618  0.918
2       CF Montreal    7    5   15   11   10   18  0.353  1.324  0.882  1.271
3         Charlotte    8   17    9   10   12   20  1.200  0.794  1.059  1.412
4      Chicago Fire    7    7    9   10   28   22  0.494  0.794  2.471  1.553
5        Cincinnati    8   14    9   10   12   15  0.988  0.794  1.059  1.059
6   Colorado Rapids   10   11   14    8    7   11  0.776  1.235  0.618  0.776
7     Columbus Crew   10   17   11    8   12   14  1.200  0.971  1.059  0.988
8         DC United   10    9   17    9    8   21  0.635  1.500  0.706  1.482
9            Dallas    7    4    9   10   18   19  0.282  0.794  1.588  1.341
10   Houston Dynamo   10   13   19    8   10    9  0.918  1.676  0.882  0.635
11      Inter Miami    9   22   15    7   14   12  1.553  1.324 

In [3]:
form_scores = {}
for idx in range(1, len(df) + 1):
    form_url = f"https://www.soccerstats.com/team.asp?league={league}&stats={idx}-"
    headers = {'User-Agent': 'Mozilla/5.0'}  #helps prevent blocking by some servers
    response = requests.get(form_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    if (response.status_code != 200):
        print(response.status_code) #error in scraping of data if code is not 200
    team = (soup.title.string).split(" statistics")[0]  #get the team from the <title> tag
    
    formNum = 12 #changeable
    formTable = soup.find_all('table')[formNum] # find the home table on the webpage
    form_df = pd.read_html(StringIO(str(formTable)))[0] # convert the html table into a DataFrame
    form_df = form_df.drop([0, form_df.index[-1], form_df.index[-2]]) #drop top row and bottom 2

    #iterate through all games to find form score
    formScore = 0
    multiplier = [2, 1.8, 1.6, 1.4, 1.2, 1]
    index = 0
    for _, row in form_df.iterrows():
        home_score, away_score = map(int, row[2][:5].split('-'))
        if row[1].startswith(team): #check if team is at home
            if home_score > away_score:
                formCounter = 3 #home win
            elif home_score == away_score:
                formCounter = 1 #home draw
            else:
                formCounter = 0 #home loss
        else:
            if home_score < away_score:
                formCounter = 3 #away win
            elif home_score == away_score:
                formCounter = 1 #away draw
            else:
                formCounter = 0 #away loss
        formScore += formCounter * multiplier[index]
        index += 1
        if index >= 6:
            break
    formScore = round(formScore / 27, 3)
    form_scores[team] = formScore
    print(team + ": " + str(formScore))
df['Form'] = df['Team'].map(form_scores)
    
#df


Inter Miami: 0.526
New York City: 0.615
Los Angeles FC: 0.659
Minnesota Utd: 0.444
Atlanta Utd: 0.37
CF Montreal: 0.311
Cincinnati: 0.437
New York RB: 0.533
Columbus Crew: 0.363
Chicago Fire: 0.622
DC United: 0.267
Toronto: 0.267
Orlando City: 0.622
Philadelphia: 0.763
Austin: 0.556
Sporting KC: 0.311
Houston Dynamo: 0.474
Dallas: 0.385
Nashville SC: 0.659
New England: 0.393
St. Louis City: 0.289
Colorado Rapids: 0.289
SJ Earthquakes: 0.385
Real Salt Lake: 0.319
Seattle: 0.378
Charlotte: 0.356
Portland: 0.533
Vancouver: 0.496
LA Galaxy: 0.319
San Diego: 0.733


In [13]:
fixt_url = f"https://www.soccerstats.com/results.asp?league={league}" ##add month num??
headers = {'User-Agent': 'Mozilla/5.0'}  #helps prevent blocking by some servers
response = requests.get(fixt_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
if (response.status_code != 200):
    print(response.status_code) #error in scraping of data if code is not 200
    
fixtNum = 9 #changeable
fixtTable = soup.find_all('table')[fixtNum] # find the home table on the webpage
fixt_df = pd.read_html(StringIO(str(fixtTable)))[0] # convert the html table into a DataFrame
fixt_df = fixt_df[~fixt_df[2].str.contains(r"^\d+ - \d+$|pp\.", na=True)].drop(columns=range(4, 10), axis=1).reset_index(drop=True)
fixt_df.reset_index(drop=True, inplace=True)
fixt_df[0] = fixt_df[0] + ' ' + fixt_df[2]
fixt_df = fixt_df.drop(columns=2, axis=1)
fixt_df.rename(columns={0: 'Date+Time', 1: 'Home Team', 3: 'Away Team'}, inplace=True)

fixt_df

,Date+Time,Home Team,Away Team
0,Th 26 Jun 00:30,CF Montreal,Cincinnati
1,Th 26 Jun 00:30,Columbus Crew,Atlanta Utd
2,Th 26 Jun 00:30,New England,Nashville SC
3,Th 26 Jun 00:30,Toronto,New York RB
4,Th 26 Jun 01:30,Chicago Fire,Philadelphia
5,Th 26 Jun 01:30,Dallas,SJ Earthquakes
6,Th 26 Jun 01:30,Minnesota Utd,Houston Dynamo
7,Th 26 Jun 01:30,Sporting KC,Charlotte
8,Th 26 Jun 01:30,St. Louis City,Orlando City
9,Th 26 Jun 02:30,Colorado Rapids,LA Galaxy


In [14]:
# Initialize columns for expected goals
fixt_df['HxG'] = 0.0
fixt_df['AxG'] = 0.0

# Loop through each row in fixt_df
for idx, row in fixt_df.iterrows():
    home_team = row['Home Team']
    away_team = row['Away Team']
    
    # Get the stats for home and away teams from df
    home_stats = df.loc[df['Team'] == home_team]
    away_stats = df.loc[df['Team'] == away_team]
    
    # Check if stats for both teams exist in df
    if not home_stats.empty and not away_stats.empty:
        # Calculate Home and Away expected goals
        home_xg = float((df['HGF'].sum() / df['HMP'].sum()) * home_stats['HS'].iloc[0]) * float(away_stats['AW'].iloc[0])
        away_xg = float((df['AGF'].sum() / df['AMP'].sum()) * away_stats['AS'].iloc[0]) * float(home_stats['HW'].iloc[0])
    else:
        # Handle missing stats
        home_xg = None
        away_xg = None
    
    # Update the fixt_df row with the calculated xG
    fixt_df.at[idx, 'HxG'] = round(home_xg, 3)
    fixt_df.at[idx, 'AxG'] = round(away_xg, 3)

fixt_df = fixt_df[['Date+Time', 'Home Team', 'HxG', 'AxG', 'Away Team']]


In [15]:
from scipy.stats import poisson

# Function to calculate win percentages
def calculate_win_percentages(row):
    home_xg = row['HxG']
    away_xg = row['AxG']
    
    # Calculate probabilities for goals 0-6
    home_probs = [poisson.pmf(k, home_xg) for k in range(7)]
    away_probs = [poisson.pmf(k, away_xg) for k in range(7)]
    
    # Calculate win probabilities
    home_win = round(sum(home_probs[i] * sum(away_probs[:i]) for i in range(1, 7)) * 100, 3)  # Home scores > Away scores
    away_win = round(sum(away_probs[i] * sum(home_probs[:i]) for i in range(1, 7)) * 100, 3)  # Away scores > Home scores
    draw     = round(sum(home_probs[i] * away_probs[i] for i in range(7)) * 100, 3)  # Home scores == Away scores
    
    return pd.Series([home_win, away_win, draw])

# Apply the function to calculate percentages and store them in new columns
fixt_df[['HW%', 'AW%', 'D%']] = fixt_df.apply(calculate_win_percentages, axis=1)

# Display the updated DataFrame
fixt_df['HF'] = fixt_df['Home Team'].map(df.set_index('Team')['Form'])
fixt_df['AF'] = fixt_df['Away Team'].map(df.set_index('Team')['Form'])
fixt_df = fixt_df[['Date+Time', 'Home Team', 'HF', 'HxG', 'HW%', 'D%', 'AW%', 'AxG', 'AF', 'Away Team']]
fixt_df


/tmp/ipykernel_1262/3057412308.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixt_df[['HW%', 'AW%', 'D%']] = fixt_df.apply(calculate_win_percentages, axis=1)
/tmp/ipykernel_1262/3057412308.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixt_df[['HW%', 'AW%', 'D%']] = fixt_df.apply(calculate_win_percentages, axis=1)
/tmp/ipykernel_1262/3057412308.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,Date+Time,Home Team,HF,HxG,HW%,D%,AW%,AxG,AF,Away Team
0,Th 26 Jun 00:30,CF Montreal,0.311,0.597,11.767,21.943,66.039,1.792,0.437,Cincinnati
1,Th 26 Jun 00:30,Columbus Crew,0.363,2.571,81.701,12.265,4.408,0.438,0.370,Atlanta Utd
2,Th 26 Jun 00:30,New England,0.393,0.382,20.233,48.189,31.578,0.547,0.659,Nashville SC
3,Th 26 Jun 00:30,Toronto,0.267,0.518,23.311,40.953,35.735,0.716,0.533,New York RB
4,Th 26 Jun 01:30,Chicago Fire,0.622,0.390,13.334,33.860,52.794,1.075,0.763,Philadelphia
5,Th 26 Jun 01:30,Dallas,0.385,0.382,15.168,38.268,46.559,0.895,0.385,SJ Earthquakes
6,Th 26 Jun 01:30,Minnesota Utd,0.444,1.145,36.826,28.808,34.333,1.095,0.474,Houston Dynamo
7,Th 26 Jun 01:30,Sporting KC,0.311,2.229,40.819,19.692,38.033,2.150,0.356,Charlotte
8,Th 26 Jun 01:30,St. Louis City,0.289,0.637,19.590,30.560,49.824,1.205,0.622,Orlando City
9,Th 26 Jun 02:30,Colorado Rapids,0.289,2.100,53.201,20.992,25.161,1.392,0.319,LA Galaxy


In [16]:
# Function to highlight min and max for 'HS' and 'HW' (default logic)
def highlight_min_max_default(column):
    min_value = column.min()
    max_value = column.max()
    
    def style_value(value):
        if value == min_value:
            return 'background-color: #ff0000; color: black'
        elif value == max_value:
            return 'background-color: #00ff00; color: black'
        else:
            return ''
    
    return column.apply(style_value)

# Function to highlight min and max for 'AS' and 'AW' (inverted logic)
def highlight_min_max_inverted(column):
    min_value = column.min()
    max_value = column.max()
    
    def style_value(value):
        if value == max_value:  # Highest value in red
            return 'background-color: #ff0000; color: black'
        elif value == min_value:  # Lowest value in green
            return 'background-color: #00ff00; color: black'
        else:
            return ''
    
    return column.apply(style_value)

# Apply different styling functions to different column groups
styled_df = df.style.apply(highlight_min_max_default, subset=['HS', 'AS', 'Form']) \
                    .apply(highlight_min_max_inverted, subset=['HW', 'AW']) \
                    .format(precision=3)

# Display the styled DataFrame (works in Jupyter)
styled_df

,Team,HMP,HGF,HGC,AMP,AGF,AGC,HS,HW,AS,AW,Form
0,Atlanta Utd,10,17,15,8,4,19,1.200,1.324,0.353,1.341,0.370
1,Austin,10,8,8,9,7,13,0.565,0.706,0.618,0.918,0.556
2,CF Montreal,7,5,15,11,10,18,0.353,1.324,0.882,1.271,0.311
3,Charlotte,8,17,9,10,12,20,1.200,0.794,1.059,1.412,0.356
4,Chicago Fire,7,7,9,10,28,22,0.494,0.794,2.471,1.553,0.622
5,Cincinnati,8,14,9,10,12,15,0.988,0.794,1.059,1.059,0.437
6,Colorado Rapids,10,11,14,8,7,11,0.776,1.235,0.618,0.776,0.289
7,Columbus Crew,10,17,11,8,12,14,1.200,0.971,1.059,0.988,0.363
8,DC United,10,9,17,9,8,21,0.635,1.500,0.706,1.482,0.267
9,Dallas,7,4,9,10,18,19,0.282,0.794,1.588,1.341,0.385
